In [103]:
def list_ru_files(dir):
    suffix='.md'
    prefix='ru-'
    import os
    rs=[]
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith(suffix) and file.startswith(prefix):
                rs.append(os.path.join(root, file))
    return rs
files=list_ru_files('/pi/langs')
print(len(files))

32


In [104]:
import re
all_words=set([])
discard_file='ru-todos.md'
files=[f for f in files if not f.endswith(discard_file)]
sentences=[]
for filename in files:
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:            
            if line.startswith(' '):
                continue
            line=line.strip()
            if len(line)==0 or line.startswith('#'):
                continue
            sentences.append(line)
            line = re.sub(r"([,.!?«»\"])", r"", line)
            all_words.update(line.split(' '))

discard_set=['', '–', '-', ',', '.']
all_words.symmetric_difference_update(x for x in all_words if x in discard_set)
all_words.symmetric_difference_update(x for x in all_words if x.isdigit())

print(len(all_words))

1338


In [36]:
import json_utils
json_utils.write_json_to_file('/pi/langs/voc/ru-voc.json', list(all_words))

In [37]:
import epitran
print(".. load rus-Cyrl")
epi = epitran.Epitran('rus-Cyrl') 

.. load rus-Cyrl


In [38]:
def get_phonetic(argument):
    # >>> print(epi.transliterate(u'Вот оно что'))
    result=epi.transliterate(argument)
    return result
get_phonetic('Вот оно что')

'vot ono t͡ɕto'

In [44]:
print(epi.transliterate('что'))
# xsampa_list: returns a list of X-SAMPA (ASCII) strings, each of which is phoneme
print(epi.xsampa_list('что'))

t͡ɕto
['ts\\', 't', 'o']


In [45]:
print(epi.xsampa_list('English'))

[]


In [47]:
print(epi.transliterate('взгляд'))
print(epi.xsampa_list('взгляд'))

vzɡlʲad
['v', 'z', 'g', 'l', 'a', 'd']


In [49]:
# epi_rev = epitran.Epitran(u'rus-Cyrl', rev=True)
# ipa='mʲod'
# tr = epi_rev.reverse_transliterate(ipa)

In [53]:
print(epi.xsampa_list('дж'))
print(epi.xsampa_list('ы'))
print(epi.xsampa_list('ц'))
print(epi.xsampa_list('ч'))

['dZ']
['1']
['ts']
['ts\\']


In [58]:
voc='''а,a
б,b
в,v
г,ɡ
д,d
е,e
ё,o
ж,ʒ
з,z
и,i
й,j
к,k
л,l
м,m
н,n
о,o
п,p
р,r
с,s
т,t
у,u
ф,f
х,x
ц,t͡s
ч,t͡ɕ
ш,ʂ
щ,ɕɕ
ъ,
ы,ɨ
ь,ʲ
э,e
ю,u
я,a
дж,d͡ʒ'''
for v in voc.split('\n'):
    pair=v.split(',')    
    print(pair[0], pair[1], '\t', epi.xsampa_list(pair[0]))

а a 	 ['a']
б b 	 ['b']
в v 	 ['v']
г ɡ 	 ['g']
д d 	 ['d']
е e 	 ['e']
ё o 	 ['o']
ж ʒ 	 ['Z']
з z 	 ['z']
и i 	 ['i']
й j 	 ['j']
к k 	 ['k']
л l 	 ['l']
м m 	 ['m']
н n 	 ['n']
о o 	 ['o']
п p 	 ['p']
р r 	 ['r']
с s 	 ['s']
т t 	 ['t']
у u 	 ['u']
ф f 	 ['f']
х x 	 ['x']
ц t͡s 	 ['ts']
ч t͡ɕ 	 ['ts\\']
ш ʂ 	 ['s`']
щ ɕɕ 	 ['s\\', 's\\']
ъ  	 []
ы ɨ 	 ['1']
ь ʲ 	 []
э e 	 ['e']
ю u 	 ['u']
я a 	 ['a']
дж d͡ʒ 	 ['dZ']


In [77]:
def get_norm_key(word, verbose=False, truncate=4):
    ipa_xs=epi.xsampa_list(word)
    if verbose:
        print('❣',word, ipa_xs)
    key=''.join(ipa_xs).replace('\\','')\
        .replace('s`', 'sh').replace("'", '')\
        .replace('1','y').lower()[:truncate]
    return key

print(get_norm_key('взгляд'))
print(get_norm_key('что'))
print(get_norm_key('Дженни'))
print(get_norm_key('мышь', True, 20))
print(get_norm_key('ошибку', True, 20))
print(get_norm_key('Лошадь', True, 20))

vzgl
tsto
dzen
❣ мышь ['m', '1', "s`'"]
mysh
❣ ошибку ['o', 's`', 'i', 'b', 'k', 'u']
oshibku
❣ Лошадь ['l', 'o', 's`', 'a', "d'"]
loshad


In [86]:
words_map={}
rev_map={}
rs=[]
for word in all_words:
    key=get_norm_key(word)
    if len(key)>0:
        if key in words_map:
            words_map[key].append(word)
        else:
            words_map[key]=[word]
        rev_map[word]=key
        rs.append('%s %s'%(key,word))
json_utils.write_json_to_file('/pi/langs/voc/ru-map.json', words_map)
print('done')

done


In [79]:
import clipboard
clipboard.copy('\n'.join(rs))

In [90]:
def word_info(word):
    ipa_xs=epi.xsampa_list(word)
    ipa=epi.transliterate(word)
    key=rev_map[word]
    print(word, ipa, ''.join(ipa_xs), '☌', key)
    print('\t', words_map[key])

word_info('мальчик')
word_info('ходят')
word_info('я')

мальчик malʲt͡ɕik mal'ts\ik ☌ malt
	 ['мальчиком', 'мальчик', 'мальчики', 'Мальчик', 'мальчиках', 'Мальчикам']
ходят xodʲat xodat ☌ xoda
	 ['ходят']
я ja a ☌ a
	 ['а', 'Я', 'я']


In [95]:
import sagas
def word_info_df(word):
    tuples=[]
    ipa_xs=epi.xsampa_list(word)
    ipa=epi.transliterate(word)
    key=rev_map[word]
    tuples.append((word, ipa, ''.join(ipa_xs), key))
    for w in words_map[key]:
        ipa_xs=epi.xsampa_list(w)
        ipa=epi.transliterate(w)
        tuples.append((w, ipa, ''.join(ipa_xs), key))
    return sagas.to_df(tuples, ['word', 'ipa', 'xsampa_list', 'key'])
word_info_df('мальчик')

,word,ipa,xsampa_list,key
0,мальчик,malʲt͡ɕik,mal'ts\ik,malt
1,мальчиком,malʲt͡ɕikom,mal'ts\ikom,malt
2,мальчик,malʲt͡ɕik,mal'ts\ik,malt
3,мальчики,malʲt͡ɕikʲi,mal'ts\iki,malt
4,Мальчик,malʲt͡ɕik,mal'ts\ik,malt
5,мальчиках,malʲt͡ɕikax,mal'ts\ikax,malt
6,Мальчикам,malʲt͡ɕikam,mal'ts\ikam,malt


In [96]:
word_info_df('я')

,word,ipa,xsampa_list,key
0,я,ja,a,a
1,а,a,a,a
2,Я,ja,a,a
3,я,ja,a,a


In [97]:
word_info_df('спрашивают')

,word,ipa,xsampa_list,key
0,спрашивают,spraʂivaut,spras`ivaut,spra
1,справа,sprava,sprava,spra
2,спрашивают,spraʂivaut,spras`ivaut,spra
3,спрашивает,spraʂivaet,spras`ivaet,spra


In [99]:
def ref_sentences(word):
    rs=[]
    for s in sentences:
        if word in s:
            rs.append(s)
    return rs

ref_sentences('спрашивают')

['Они спрашивают, где коробка.']

In [102]:
from sagas.nlu.nlu_tools import NluTools
def say_refs(word):
    sents=ref_sentences(word)
    for s in sents:
        print(s)
        NluTools().say(s, 'ru')
say_refs('спрашивают')

Они спрашивают, где коробка.
speak language with ru ...
done.


⊕ [Pandas Tutorial 1: Pandas Basics (read_csv, DataFrame, Data Selection)](https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/)

⊕ [Pandas: Select rows that match a string - David Hamann](https://davidhamann.de/2017/06/26/pandas-select-elements-by-string/)

⊕ [python - Ignoring NaNs with str.contains - Stack Overflow](https://stackoverflow.com/questions/28311655/ignoring-nans-with-str-contains)


In [106]:
import pandas as pd
verbs='/pi/nlp/russian-dictionary/verbs.csv'
df=pd.read_csv(verbs, delimiter = '\t')
df[:5]

,bare,accented,translations_en,translations_de,aspect,partner,imperative_sg,imperative_pl,past_m,past_f,past_n,past_pl,presfut_sg1,presfut_sg2,presfut_sg3,presfut_pl1,presfut_pl2,presfut_pl3
0,быть,быть,be,"sein, werden",imperfective,NaN,бу'дь,бу'дьте,бы'л,была',бы'ло,бы'ли,есть,есть,есть,есть,есть,есть
1,сказать,сказа'ть,"say, tell, speak, talk, indicate, point, be ev...","sagen, mitteilen",perfective,говорить,скажи',скажи'те,сказа'л,сказа'ла,сказа'ло,сказа'ли,скажу',ска'жешь,ска'жет,ска'жем,ска'жете,ска'жут
2,мочь,мочь,"be able, can","können, imstande sein, dürfen",NaN,NaN,моги',моги'те,мо'г,могла',могло',могли',могу',мо'жешь,мо'жет,мо'жем,мо'жете,мо'гут
3,знать,знать,"know, be aware, be acquainted, have a knowledg...","wissen, kennen, sich auskennen, Bescheid wisse...",imperfective,узнать,зна'й,зна'йте,зна'л,зна'ла,зна'ло,зна'ли,зна'ю,зна'ешь,зна'ет,зна'ем,зна'ете,зна'ют
4,говорить,говори'ть,"say, tell, speak, talk, indicate, point, be ev...","sprechen, reden, sagen",imperfective,сказать,говори',говори'те,говори'л,говори'ла,говори'ло,говори'ли,говорю',говори'шь,говори'т,говори'м,говори'те,говоря'т


In [117]:
# select all rows where the column ‘model’ starts with the string
df[df['bare'].str.match('сказать')]

,bare,accented,translations_en,translations_de,aspect,partner,imperative_sg,imperative_pl,past_m,past_f,past_n,past_pl,presfut_sg1,presfut_sg2,presfut_sg3,presfut_pl1,presfut_pl2,presfut_pl3
1,сказать,сказа'ть,"say, tell, speak, talk, indicate, point, be ev...","sagen, mitteilen",perfective,говорить,скажи',скажи'те,сказа'л,сказа'ла,сказа'ло,сказа'ли,скажу',ска'жешь,ска'жет,ска'жем,ска'жете,ска'жут
2516,сказаться,сказа'ться,"tell, report oneself","sich zeigen, zum Ausdruck kommen, sich auswirken",NaN,NaN,скажи'сь,скажи'тесь,сказа'лся,сказа'лась,сказа'лось,сказа'лись,скажу'сь,ска'жешься,ска'жется,ска'жемся,ска'жетесь,ска'жутся


In [118]:
df[df['translations_en'].str.contains('address', na=False)]

,bare,accented,translations_en,translations_de,aspect,partner,imperative_sg,imperative_pl,past_m,past_f,past_n,past_pl,presfut_sg1,presfut_sg2,presfut_sg3,presfut_pl1,presfut_pl2,presfut_pl3
251,обратиться,обрати'ться,"appeal, apply, address, turn, treat, handle, u...","sich wenden, sich richten, anreden",perfective,обращаться,обрати'сь,обрати'тесь,обрати'лся,обрати'лась,обрати'лось,обрати'лись,обращу'сь,обрати'шься,обрати'тся,обрати'мся,обрати'тесь,обратя'тся
270,обращаться,обраща'ться,"appeal, apply, address, turn, treat, handle, u...","umgehen, behandeln, handhaben; sich wenden an",imperfective,обратиться,обраща'йся,обраща'йтесь,обраща'лся,обраща'лась,обраща'лось,обраща'лись,обраща'юсь,обраща'ешься,обраща'ется,обраща'емся,обраща'етесь,обраща'ются
2924,адресовать,адресова'ть,"address, direct",richten,NaN,NaN,адресу'й,адресу'йте,адресова'л,адресова'ла,адресова'ло,адресова'ли,адресу'ю,адресу'ешь,адресу'ет,адресу'ем,адресу'ете,адресу'ют
3464,заговаривать,загова'ривать,"speak, address, accost, talk head off, tire ou...",NaN,imperfective,заговорить,загова'ривай,загова'ривайте,загова'ривал,загова'ривала,загова'ривало,загова'ривали,загова'риваю,загова'риваешь,загова'ривает,загова'риваем,загова'риваете,загова'ривают
6661,напутствовать,напу'тствовать,address,NaN,NaN,NaN,напу'тствуй,напу'тствуйте,напу'тствовал,напу'тствовала,напу'тствовало,напу'тствовали,напу'тствую,напу'тствуешь,напу'тствует,напу'тствуем,напу'тствуете,напу'тствуют
7614,адресоваться,адресова'ться,address oneself,NaN,NaN,NaN,адресу'йся,адресу'йтесь,адресова'лся,адресова'лась,адресова'лось,адресова'лись,адресу'юсь,адресу'ешься,адресу'ется,адресу'емся,адресу'етесь,адресу'ются
11060,переадресовывать,переадресо'вывать,readdress,NaN,NaN,NaN,переадресо'вывай,переадресо'вывайте,переадресо'вывал,переадресо'вывала,переадресо'вывало,переадресо'вывали,переадресо'вываю,переадресо'вываешь,переадресо'вывает,переадресо'вываем,переадресо'вываете,переадресо'вывают


⊕ [Dealing with Rows and Columns in Pandas DataFrame - GeeksforGeeks](https://www.geeksforgeeks.org/dealing-with-rows-and-columns-in-pandas-dataframe/)


In [137]:
rs_df=df[df['translations_en'].str.contains('address', na=False)]
rs_df[['accented', 'translations_en']]

,accented,translations_en
251,обрати'ться,"appeal, apply, address, turn, treat, handle, u..."
270,обраща'ться,"appeal, apply, address, turn, treat, handle, u..."
2924,адресова'ть,"address, direct"
3464,загова'ривать,"speak, address, accost, talk head off, tire ou..."
6661,напу'тствовать,address
7614,адресова'ться,address oneself
11060,переадресо'вывать,readdress


In [128]:
# rs_df=df['be' in df['translations_en'].str.split(', ')]
# rs_df[['accented', 'translations_en']].head()

In [136]:
rs_df=df[df['translations_en'].str.contains('be,', na=False)]
rs_df[['accented', 'translations_en']].head()

,accented,translations_en
9,стоя'ть,"stand, be, lie, be situated, stop, be stagnant..."
16,сиде'ть,"sit, be perched, be, fit"
27,вы'йти,"go out, alight, get out, appear, be published,..."
30,оказа'ться,"to turn out to be, to be found to be"
38,лежа'ть,"lie; be, be situated"


⊕ [pandas.Series.str.contains — pandas 0.23.3 documentation](https://pandas.pydata.org/pandas-docs/version/0.23.3/generated/pandas.Series.str.contains.html)


In [144]:
rs_df=df[df['translations_en'].str.contains('thank|appreciate', flags=re.IGNORECASE, regex=True, na=False)]
rs_df[['accented', 'translations_en']].head()

,accented,translations_en
819,благодари'ть,thank
1006,цени'ть,"value, estimate, appreciate"
1425,поблагодари'ть,thank
9003,возблагодари'ть,render thanks


In [153]:
for index, row in rs_df.iterrows():
    print(row['accented'], row['translations_en'], row['partner'])
    print('', pd.notnull(row['partner']))
    # print(row.keys)
    # print('abs' in row)

благодари'ть thank поблагодарить;отблагодарить
 True
цени'ть value, estimate, appreciate оценить
 True
поблагодари'ть thank благодарить
 True
возблагодари'ть render thanks nan
 False


In [159]:
print(len(rs_df))
print(len(df))

4
13909


In [165]:
cols='aspect	partner	imperative_sg	imperative_pl	past_m	past_f	past_n	past_pl	presfut_sg1	presfut_sg2	presfut_sg3	presfut_pl1	presfut_pl2	presfut_pl3'
cols_list=cols.split('\t')
print(cols_list)

def append_cols(key, row, cols_list):
    rs=[]
    for col in cols_list:
        if pd.notnull(row[col]):
            rs.append('%s %s'%(key, row[col].replace("'",'')))
    return rs

['aspect', 'partner', 'imperative_sg', 'imperative_pl', 'past_m', 'past_f', 'past_n', 'past_pl', 'presfut_sg1', 'presfut_sg2', 'presfut_sg3', 'presfut_pl1', 'presfut_pl2', 'presfut_pl3']


In [170]:
def norm_key(s):
    s = re.sub(r"[^a-zA-Z]+", r"", s)
    return s[:4].lower()
norm_key('re nder ok')

'rend'

In [171]:
# import sagas
import json

def build_pairs(rs_df):
    input_pairs=[]
    for index, row in rs_df.iterrows():
        if pd.notnull(row['translations_en']):
            ru_word=row['bare']
            en_words=row['translations_en'].split(', ')
            for en_word in en_words:
                key=norm_key(en_word)
                input_pairs.append('%s %s'%(key, ru_word))
                input_pairs.extend(append_cols(key, row, cols_list))
    return input_pairs

input_pairs=build_pairs(rs_df)
print(json.dumps(input_pairs, indent=2, ensure_ascii=False))

[
  "than благодарить",
  "than imperfective",
  "than поблагодарить;отблагодарить",
  "than благодари",
  "than благодарите",
  "than благодарил",
  "than благодарила",
  "than благодарило",
  "than благодарили",
  "than благодарю",
  "than благодаришь",
  "than благодарит",
  "than благодарим",
  "than благодарите",
  "than благодарят",
  "valu ценить",
  "valu imperfective",
  "valu оценить",
  "valu цени",
  "valu цените",
  "valu ценил",
  "valu ценила",
  "valu ценило",
  "valu ценили",
  "valu ценю",
  "valu ценишь",
  "valu ценит",
  "valu ценим",
  "valu цените",
  "valu ценят",
  "esti ценить",
  "esti imperfective",
  "esti оценить",
  "esti цени",
  "esti цените",
  "esti ценил",
  "esti ценила",
  "esti ценило",
  "esti ценили",
  "esti ценю",
  "esti ценишь",
  "esti ценит",
  "esti ценим",
  "esti цените",
  "esti ценят",
  "appr ценить",
  "appr imperfective",
  "appr оценить",
  "appr цени",
  "appr цените",
  "appr ценил",
  "appr ценила",
  "appr ценило",
  "appr цен

In [168]:
print('\n'.join(input_pairs))

than благодарить
than imperfective
than поблагодарить;отблагодарить
than благодари
than благодарите
than благодарил
than благодарила
than благодарило
than благодарили
than благодарю
than благодаришь
than благодарит
than благодарим
than благодарите
than благодарят
valu ценить
valu imperfective
valu оценить
valu цени
valu цените
valu ценил
valu ценила
valu ценило
valu ценили
valu ценю
valu ценишь
valu ценит
valu ценим
valu цените
valu ценят
esti ценить
esti imperfective
esti оценить
esti цени
esti цените
esti ценил
esti ценила
esti ценило
esti ценили
esti ценю
esti ценишь
esti ценит
esti ценим
esti цените
esti ценят
appr ценить
appr imperfective
appr оценить
appr цени
appr цените
appr ценил
appr ценила
appr ценило
appr ценили
appr ценю
appr ценишь
appr ценит
appr ценим
appr цените
appr ценят
than поблагодарить
than perfective
than благодарить
than поблагодари
than поблагодарите
than поблагодарил
than поблагодарила
than поблагодарило
than поблагодарили
than поблагодарю
than поблагодаришь


In [173]:
import clipboard
input_pairs=build_pairs(df)
print(len(input_pairs))

442281


In [ ]:
clipboard.copy('\n'.join(input_pairs))
print('ok')

In [174]:
'Какой'.lower()

'какой'

In [175]:
rs_df

,bare,accented,translations_en,translations_de,aspect,partner,imperative_sg,imperative_pl,past_m,past_f,past_n,past_pl,presfut_sg1,presfut_sg2,presfut_sg3,presfut_pl1,presfut_pl2,presfut_pl3
819,благодарить,благодари'ть,thank,"danken, sich bedanken für",imperfective,поблагодарить;отблагодарить,благодари',благодари'те,благодари'л,благодари'ла,благодари'ло,благодари'ли,благодарю',благодари'шь,благодари'т,благодари'м,благодари'те,благодаря'т
1006,ценить,цени'ть,"value, estimate, appreciate","schätzen, würdigen",imperfective,оценить,цени',цени'те,цени'л,цени'ла,цени'ло,цени'ли,ценю',це'нишь,це'нит,це'ним,це'ните,це'нят
1425,поблагодарить,поблагодари'ть,thank,"danken, sich bedanken",perfective,благодарить,поблагодари',поблагодари'те,поблагодари'л,поблагодари'ла,поблагодари'ло,поблагодари'ли,поблагодарю',поблагодари'шь,поблагодари'т,поблагодари'м,поблагодари'те,поблагодаря'т
9003,возблагодарить,возблагодари'ть,render thanks,sich bedanken,NaN,NaN,возблагодари',возблагодари'те,возблагодари'л,возблагодари'ла,возблагодари'ло,возблагодари'ли,возблагодарю',возблагодари'шь,возблагодари'т,возблагодари'м,возблагодари'те,возблагодаря'т


In [180]:
s=set([])
s.add('first')
s.update(['second','third'])
s

{'first', 'second', 'third'}

In [181]:
'оценить'.split(';')

['оценить']

In [204]:
import pandas as pd
verbs='/pi/nlp/russian-dictionary/adjectives.csv'
df=pd.read_csv(verbs, delimiter = '\t')
df[df['incomparable'].notnull()].head()

,bare,accented,position,translations_en,translations_de,incomparable,comparative,superlative,short_m,short_f,...,decl_n_dat,decl_n_acc,decl_n_inst,decl_n_prep,decl_pl_nom,decl_pl_gen,decl_pl_dat,decl_pl_acc,decl_pl_inst,decl_pl_prep
7,большой,большо'й,NaN,"big, large, great, important, grown-up",groß,0.0,бо'льше;бо'льший,наибо'льший,вели'к,велика',...,большо'му,большо'е,больши'м,большо'м,больши'е,больши'х,больши'м,"больши'х,больши'е",больши'ми,больши'х
13,новый,но'вый,NaN,"new, novel, modern, fresh",neu,0.0,нове'е,нове'йший,нов,нова',...,но'вому,"но'вое, но'вого",но'вым,но'вом,но'вые,но'вых,но'вым,"но'вые, но'вых",но'выми,но'вых
15,хороший,хоро'ший,NaN,"good, beautiful, handsome","gut, schön",0.0,лу'чше,лу'чший;наилу'чший,хоро'ш,хороша',...,хоро'шему,хоро'шее,хоро'шим,хоро'шем,хоро'шие,хоро'ших,хоро'шим,"хоро'ших,хоро'шие",хоро'шими,хоро'ших
19,старый,ста'рый,NaN,"old, ancient, the old, the past","alt, der frühere",0.0,ста'рше;старе'е,са'мый ста'рый,стар,стара',...,ста'рому,ста'рое,ста'рым,ста'ром,ста'рые,ста'рых,ста'рым,"ста'рых, ста'рые",ста'рыми,ста'рых
22,главный,гла'вный,NaN,"main, chief, principal, head","Chef-, Haupt-",0.0,главне'е,са'мый гла'вный,гла'вен,главна',...,гла'вному,гла'вное,гла'вным,гла'вном,гла'вные,гла'вных,гла'вным,"гла'вные,гла'вных",гла'вными,гла'вных


In [253]:
import pandas as pd
verbs='/pi/nlp/russian-dictionary/nouns.csv'
df=pd.read_csv(verbs, delimiter = '\t')
df[df['partner'].notnull()].head()

,bare,accented,translations_en,translations_de,gender,partner,animate,indeclinable,sg_only,pl_only,...,sg_dat,sg_acc,sg_inst,sg_prep,pl_nom,pl_gen,pl_dat,pl_acc,pl_inst,pl_prep
65,бог,бог,"God, god",Gott,m,богиня,1.0,0.0,0.0,0.0,...,бо'гу,бо'га,бо'гом,бо'ге,бо'ги,бого'в,бога'м,бого'в,бога'ми,бога'х
82,старик,стари'к,old man,"der Alte, alter Mann",m,старуха,1.0,0.0,0.0,0.0,...,старику',старика',старико'м,старике',старики',старико'в,старика'м,старико'в,старика'ми,старика'х
90,начальник,нача'льник,"head, chief, superior, boss","Chef, der Vorgesetzte, Leiter",m,начальница,1.0,0.0,0.0,0.0,...,нача'льнику,нача'льника,нача'льником,нача'льнике,нача'льники,нача'льников,нача'льникам,нача'льников,нача'льниками,нача'льниках
132,немец,не'мец,German; German man,der Deutsche,m,немка,1.0,0.0,0.0,0.0,...,не'мцу,не'мца,не'мцем,не'мце,не'мцы,не'мцев,не'мцам,не'мцев,не'мцами,не'мцах
146,гость,гость,"visitor, guest",Gast,m,гостья,1.0,0.0,0.0,0.0,...,го'стю,го'стя,го'стем,го'сте,го'сти,госте'й,гостя'м,госте'й,гостя'ми,гостя'х


In [255]:
def search_en(word_pat):
    return df[df['translations_en'].str.contains(word_pat, flags=re.IGNORECASE, regex=True, na=False)]
rs_words=search_en('canada')
rs_words

,bare,accented,translations_en,translations_de,gender,partner,animate,indeclinable,sg_only,pl_only,...,sg_dat,sg_acc,sg_inst,sg_prep,pl_nom,pl_gen,pl_dat,pl_acc,pl_inst,pl_prep
2249,Канада,Кана'да,Canada,Kanada,f,NaN,NaN,0.0,1.0,0.0,...,Кана'де,Кана'ду,"Кана'дой, Кана'дою",Кана'де,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
for index, row in rs_words.iterrows():
    if pd.notnull(row['translations_en']):
        en_words=row['translations_en'].split(', ')
        print(en_words)

['Canada']


In [200]:
cols='bare	partner	imperative_sg	imperative_pl	past_m	past_f	past_n	past_pl	presfut_sg1	presfut_sg2	presfut_sg3	presfut_pl1	presfut_pl2	presfut_pl3'
cols_list=cols.split('\t')
print(cols_list)

def process_ru_cols(row, cols_list):
    rs=set([])
    for col in cols_list:
        if pd.notnull(row[col]):
            rs.update(row[col].replace("'",'').split(';'))
    return list(rs)
def add_to(rs_df, all_dicts, cols_list):
    for index, row in rs_df.iterrows():
        if pd.notnull(row['translations_en']):
            ru_word=row['bare']
            en_words=row['translations_en'].split(', ')
            if pd.notnull(row['translations_de']):
                de_words=row['translations_de'].split(', ')
            else:
                de_words=[]
            all_dicts.append({'en':en_words, 
                              'ru':process_ru_cols(row, cols_list),
                              'de':de_words
                             })
def print_json(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))
    
all_dicts=[]
add_to(rs_df, all_dicts, cols_list)
print_json(all_dicts)

['bare', 'partner', 'imperative_sg', 'imperative_pl', 'past_m', 'past_f', 'past_n', 'past_pl', 'presfut_sg1', 'presfut_sg2', 'presfut_sg3', 'presfut_pl1', 'presfut_pl2', 'presfut_pl3']
[
  {
    "en": [
      "thank"
    ],
    "ru": [
      "благодарил",
      "отблагодарить",
      "благодарили",
      "благодарим",
      "благодарить",
      "благодарите",
      "благодарила",
      "благодарю",
      "благодаришь",
      "благодарят",
      "благодарило",
      "поблагодарить",
      "благодари",
      "благодарит"
    ],
    "de": [
      "danken",
      "sich bedanken für"
    ]
  },
  {
    "en": [
      "value",
      "estimate",
      "appreciate"
    ],
    "ru": [
      "оценить",
      "цените",
      "ценили",
      "ценил",
      "ценю",
      "ценишь",
      "ценим",
      "ценило",
      "ценит",
      "ценила",
      "ценить",
      "ценят",
      "цени"
    ],
    "de": [
      "schätzen",
      "würdigen"
    ]
  },
  {
    "en": [
      "thank"
    ],
    "ru": [
  

In [239]:
def filter_set(all_dicts, ru_word_set, lang='ru', absents=None):
    rs=[]
    for word in ru_word_set:
        word=word.lower()
        find_it=False
        for item in all_dicts:
            if word in item[lang]:
                rs.append(item)
                find_it=True
        if absents is not None and not find_it:
            absents.append(word)            
    return rs

skips=[]
filter_set(all_dicts, ['благодарить', 'ценил', 'nob'], 'ru', skips)

[{'de': ['danken', 'sich bedanken für'],
  'en': ['thank'],
  'ru': ['благодарил',
   'отблагодарить',
   'благодарили',
   'благодарим',
   'благодарить',
   'благодарите',
   'благодарила',
   'благодарю',
   'благодаришь',
   'благодарят',
   'благодарило',
   'поблагодарить',
   'благодари',
   'благодарит']},
 {'de': ['danken', 'sich bedanken'],
  'en': ['thank'],
  'ru': ['поблагодари',
   'поблагодарила',
   'благодарить',
   'поблагодаришь',
   'поблагодарят',
   'поблагодарит',
   'поблагодарите',
   'поблагодарил',
   'поблагодарили',
   'поблагодарим',
   'поблагодарить',
   'поблагодарю',
   'поблагодарило']},
 {'de': ['schätzen', 'würdigen'],
  'en': ['value', 'estimate', 'appreciate'],
  'ru': ['оценить',
   'цените',
   'ценили',
   'ценил',
   'ценю',
   'ценишь',
   'ценим',
   'ценило',
   'ценит',
   'ценила',
   'ценить',
   'ценят',
   'цени']}]

In [240]:
print(len(skips), skips)

1 ['nob']


In [210]:
all_dicts=[]
# dict_maps={'v':'verbs', 'n':'nouns', 'a':'adjectives', 'o':'others'}
dict_maps={'verbs':cols_list,
           'nouns':'bare	partner	sg_nom	sg_gen	sg_dat	sg_acc	sg_inst	sg_prep	pl_nom	pl_gen	pl_dat	pl_acc	pl_inst	pl_prep'.split('\t'),
           'adjectives':'bare	comparative	superlative	short_m	short_f	short_n	short_pl	decl_m_nom	decl_m_gen	decl_m_dat	decl_m_acc	decl_m_inst	decl_m_prep	decl_f_nom	decl_f_gen	decl_f_dat	decl_f_acc	decl_f_inst	decl_f_prep	decl_n_nom	decl_n_gen	decl_n_dat	decl_n_acc	decl_n_inst	decl_n_prep	decl_pl_nom	decl_pl_gen	decl_pl_dat	decl_pl_acc	decl_pl_inst	decl_pl_prep'.split('\t'),
           'others':'bare'.split('\t'),           
          }
for dict_name, cols in dict_maps.items():
    verbs = '/pi/nlp/russian-dictionary/%s.csv'%dict_name
    print('process', verbs)
    df = pd.read_csv(verbs, delimiter='\t')
    add_to(df, all_dicts, cols)
print('total', len(all_dicts))

process /pi/nlp/russian-dictionary/verbs.csv
process /pi/nlp/russian-dictionary/nouns.csv
process /pi/nlp/russian-dictionary/adjectives.csv
process /pi/nlp/russian-dictionary/others.csv
total 44747


In [211]:
print('играть'.lower())
filter_set(all_dicts, ['играть'])

играть


[{'de': ['spielen', 'vorspielen', 'mitwirken'],
  'en': ['play', 'act', 'perform', 'sparkle', 'toy', 'trifle'],
  'ru': ['играем',
   'сыграть',
   'играете',
   'играйте',
   'играло',
   'поиграть',
   'играй',
   'играла',
   'играли',
   'играть',
   'играют',
   'играю',
   'играешь',
   'играет',
   'играл']},
 {'de': ['vorspielen'],
  'en': ['play', 'act', 'perform', 'sparkle', 'toy', 'trifle'],
  'ru': ['сыграй',
   'сыграл',
   'сыграть',
   'сыграла',
   'сыграют',
   'сыграем',
   'сыграете',
   'играть',
   'сыграю',
   'сыграйте',
   'сыграли',
   'сыграет',
   'сыграешь',
   'сыграло']}]

In [212]:
print('Какой'.lower())
filter_set(all_dicts, ['Какой'])

какой


[{'de': ['welcher', 'was für', 'was für ein', 'welch'],
  'en': ['what', 'how', 'which', 'what kind of'],
  'ru': ['какие',
   'каких',
   'каким',
   'какой,какого',
   'какими',
   'каком',
   'какую',
   'какое',
   'какой',
   'какого',
   'какому',
   'какие,каких',
   'какой, какою',
   'какая']}]

In [214]:
filter_set(all_dicts, ['bird'], 'en')

[{'de': ['Vogel'],
  'en': ['bird'],
  'ru': ['птицами',
   'птицы',
   'птицей, птицею',
   'птицах',
   'птиц',
   'птицам',
   'птице',
   'птицу',
   'птица']},
 {'de': ['Vogel-'],
  'en': ['bird', 'avian', 'poultry'],
  'ru': ['птичьем',
   'птичьей',
   'птичьего',
   'птичью',
   'птичьих,птичьи',
   'птичьему',
   'птичьими',
   'птичья',
   'птичьих',
   'птичьего,птичий',
   'птичье',
   'птичьим',
   'птичьи',
   'птичьей,птичьею',
   'птичий']}]

In [245]:
filter_set(all_dicts, ['С'], 'ru')

[{'de': ['mit', 'und', 'von', 'aus', 'seit', 'ab', 'vor', 'etwa', 'ungefähr'],
  'en': ['with',
   'and',
   'from',
   'of',
   'since',
   'beginning from',
   'in',
   'on',
   'at',
   'the size of',
   'about'],
  'ru': ['с']}]

In [250]:
filter_set(all_dicts, ['Канада'], 'ru')

[]

In [248]:
filter_set(all_dicts, ['canada'], 'en')

[]

In [228]:
len(all_dicts)

44747

In [226]:
filter_set(all_dicts, ['мы','нами'], 'ru')

[]

In [227]:
filter_set(all_dicts, ['we','us'], 'en')

[]

In [241]:
import json_utils
voc_file='/pi/langs/voc/ru-voc.json'
voc_dicts='/pi/langs/voc/ru-voc-dicts.json'
words=json_utils.read_json_file(voc_file)
skips=[]
rs=filter_set(all_dicts, words, 'ru', skips)
json_utils.write_json_to_file(voc_dicts, rs)
print('done.')
print('absents', len(skips))

done.
absents 137


In [243]:
json_utils.write_json_to_file('/pi/langs/voc/ru-voc-absents.json', skips)

In [223]:
voc_words=json_utils.read_json_file(voc_dicts)
filter_set(voc_words, ['С'], 'ru')

[{'de': ['mit', 'und', 'von', 'aus', 'seit', 'ab', 'vor', 'etwa', 'ungefähr'],
  'en': ['with',
   'and',
   'from',
   'of',
   'since',
   'beginning from',
   'in',
   'on',
   'at',
   'the size of',
   'about'],
  'ru': ['с']},
 {'de': ['mit', 'und', 'von', 'aus', 'seit', 'ab', 'vor', 'etwa', 'ungefähr'],
  'en': ['with',
   'and',
   'from',
   'of',
   'since',
   'beginning from',
   'in',
   'on',
   'at',
   'the size of',
   'about'],
  'ru': ['с']}]

In [225]:
filter_set(voc_words, ['мы','нами'], 'ru')

[]

In [219]:
filter_set(voc_words, ['girl'], 'en')

[{'de': ['Mädchen'],
  'en': ['girl', 'little girl', 'girlie'],
  'ru': ['девочка',
   'девочкам',
   'девочками',
   'девочек',
   'девочках',
   'девочки',
   'девочкой, девочкою',
   'девочке',
   'девочку']},
 {'de': ['Mädchen'],
  'en': ['girl', 'little girl', 'girlie'],
  'ru': ['девочка',
   'девочкам',
   'девочками',
   'девочек',
   'девочках',
   'девочки',
   'девочкой, девочкою',
   'девочке',
   'девочку']},
 {'de': ['Mädchen'],
  'en': ['girl', 'little girl', 'girlie'],
  'ru': ['девочка',
   'девочкам',
   'девочками',
   'девочек',
   'девочках',
   'девочки',
   'девочкой, девочкою',
   'девочке',
   'девочку']},
 {'de': ['Mädchen'],
  'en': ['girl', 'little girl', 'girlie'],
  'ru': ['девочка',
   'девочкам',
   'девочками',
   'девочек',
   'девочках',
   'девочки',
   'девочкой, девочкою',
   'девочке',
   'девочку']},
 {'de': ['Mädchen', 'Fräulein', 'junge Frau; Partnerin', 'Freundin'],
  'en': ['girl', 'lass', 'miss'],
  'ru': ['девушке',
   'девушках',
   'девуш

In [221]:
print(len(voc_words))
rs=[]
for w in voc_words:
    for en_w in w['en']:
        ru_w=' '.join(w['ru']).replace(',','')
        rs.append('%s %s'%(norm_key(en_w), ru_w))
print(len(rs))

1396
4911


In [222]:
clipboard.copy('\n'.join(rs))

In [244]:
['x']+['y','z']

['x', 'y', 'z']